In [1]:
from halotools.sim_manager import CachedHaloCatalog, FakeSim
from halotools.empirical_models import PrebuiltHodModelFactory, Zheng07Cens, Zheng07Sats, TrivialPhaseSpace, NFWPhaseSpace, HodModelFactory
from halotools.mock_observables import return_xyz_formatted_array
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import time
from multiprocessing import Pool, cpu_count
import emcee
import corner
from Corrfunc.theory.wp import wp
import MCMC_data_file
from numpy.linalg import inv
import scipy.optimize as op

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [97]:
wp_ng_vals = MCMC_data_file.get_wp()
bin_edges = MCMC_data_file.get_bins()
bin_cen = (bin_edges[1:]+bin_edges[:-1])/2.
wp_vals = wp_ng_vals[1:len(wp_ng_vals)]
ng = wp_ng_vals[0]
cov_matrix = MCMC_data_file.get_cov()
invcov = inv(cov_matrix[1:len(cov_matrix),1:len(cov_matrix)])
ng_cov = cov_matrix[0,0]

In [2]:
cens_occ_model = Zheng07Cens()
cens_prof_model = TrivialPhaseSpace()

sats_occ_model =  Zheng07Sats()
sats_prof_model = NFWPhaseSpace()

In [78]:
def resid(avec, bin_cen, wp_vals):
    """ the residual function -- this is what will be minimized by the
        scipy.optimize.leastsq() routine.  avec is the parameters we
        are optimizing -- they are packed in here, so we unpack to
        begin.  (x, y) are the data points 

        scipy.optimize.leastsq() minimizes:

           x = arg min(sum(func(y)**2,axis=0))
                    y

        so this should just be the distance from a point to the curve,
        and it will square it and sum over the points
        """

    logMmin, sigma_logM, alpha, logM0, logM1 = avec

    # note: if we wanted to deal with error bars, we would weight each
    # residual accordingly
    
    pi_max = 60.
    Lbox = 250.
    
    cens_occ_model.param_dict['logMmin'] = logMmin
    cens_occ_model.param_dict['sigma_logM'] = sigma_logM
    sats_occ_model.param_dict['alpha'] = alpha
    sats_occ_model.param_dict['logM0'] = logM0
    sats_occ_model.param_dict['logM1'] = logM1

    model_instance = HodModelFactory(centrals_occupation = cens_occ_model, centrals_profile = cens_prof_model, 
                                 satellites_occupation = sats_occ_model, satellites_profile = sats_prof_model)

    halocat = CachedHaloCatalog(simname='bolplanck',redshift = 0.0) 
    model_instance.populate_mock(halocat) 
    mvir_table = model_instance.mock.halo_table

    pos = return_xyz_formatted_array(model_instance.mock.galaxy_table['x'], model_instance.mock.galaxy_table['y'],
                                 model_instance.mock.galaxy_table['z'],period = Lbox)
    x = pos[:,0]
    y = pos[:,1]
    z = pos[:,2]
    velz = model_instance.mock.galaxy_table['vz']
    pos_zdist = return_xyz_formatted_array(x,y,z, period = Lbox, velocity=velz, velocity_distortion_dimension='z')
    

    wp_calc = wp(Lbox,pi_max,1,bin_edges,pos_zdist[:,0],pos_zdist[:,1],pos_zdist[:,2],
                    verbose=True,xbin_refine_factor=2, ybin_refine_factor=2, zbin_refine_factor=1)
    
    wp_diff = wp_vals-wp_calc['wp']
    ng_diff = ng -  model_instance.mock.number_density
    
    x2= -0.5 * np.dot(wp_diff, np.dot(invcov, wp_diff)) + -0.5 * (ng_diff**2) * ng_cov
    return x2

In [56]:
#Guess
logMmin = 11.96
sigma_logM = 0.38
alpha = 1.16
logM0 = 12.23
logM1 = 13.28

In [63]:
afit = op.least_squares(resid, [logMmin, sigma_logM, alpha, logM0, logM1],args=(bin_cen, wp_vals),
                        ftol=1e-02, xtol=1e-02, gtol=1e-02,)

In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 15,15,4.  Time taken =   0.006 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.265 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 15,15,4.  Time taken =   0.008 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.191 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 15,15,4.  Time taken =   0.005 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........100% done. Time taken =  2.185 secs
In gridlink_index_particles_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 15,15,4.  Time taken =   0.005 sec
0%.........10%.........20%.........30%.........40%.........50%.........60%.....

In [64]:
print(afit.x)

[11.95999965  0.38000001  1.16000005 12.22999995 13.28000016]
